In [28]:
%matplotlib inline
import ast
from bs4 import BeautifulSoup
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import re
import requests
from scipy import stats
import seaborn as sns
import sqlite3
from sqlite3 import Error
import time
import datetime
from config import api_key_tmdb

In [45]:
# Importing the IMDB movies file
data = "/Users/florencialeoni/Documents/flatironDS/Mod_1_FP/mod1_movies/imdb_movies_data.csv"
use_cols = ["imdb_id", "title", "imdb_score", "meta_score", "year"]

# Using pandas.read_csv method to load the file
imdb_df = pd.read_csv(data, usecols = use_cols)

In [41]:
# Replace missing data in the "imdb_score" and "meta_score" columns with NumPy NaN values
imdb_df["imdb_score"] = imdb_df["imdb_score"].replace(" ", np.nan)
imdb_df["meta_score"] = imdb_df["meta_score"].replace(" ", np.nan)

In [59]:
# Importing the TMDB Movies file
data = "/Users/florencialeoni/Documents/flatironDS/Mod_1_FP/mod1_movies/tmdb_movies_cast_df.csv"
use_cols = ["id", "imdb_id", "title", "belongs_to_collection", "budget", "genres",  "overview", "release_date", 
                   "revenue", "tagline", "vote_average", "vote_count", "runtime", "popularity", "production_companies"]

# Using pandas.read_csv method to load the file
tmdb_df = pd.read_csv(data, lineterminator='\n')
tmdb_df = tmdb_df.set_index("id")


In [70]:
# Using pandas.to_numeric to coerce the "id", "budget" and "revenue" columns into numbers
tmdb_df["budget"] = pd.to_numeric(tmdb_df["budget"], downcast = "integer", errors = "coerce")
tmdb_df["revenue"] = pd.to_numeric(tmdb_df["revenue"], downcast = "integer", errors = "coerce")

# Lambda function to strip and split "release_date" string values
f = lambda x: str(x).strip().split('-')[0] if x != np.nan else np.nan

# Creating a new column, "year", and assigninf the value of "release_date" 
# After applying the lambda function through map
tmdb_df["year"] = tmdb_df["release_date"].map(f)

# Coerce the "year" column to integers
tmdb_df["year"] = pd.to_numeric(tmdb_df["year"], errors = "coerce")

# Convert "release_date" column to datetime format
tmdb_df["release_date"] = pd.to_datetime(tmdb_df["release_date"])

In [71]:
# Divide "release_date" column in days and months
tmdb_df["month_of_year"] = tmdb_df["release_date"].dt.month_name()
tmdb_df["day_of_week"] = tmdb_df["release_date"].dt.day_name()

In [74]:
tmdb_df = tmdb_df.drop(["release_date"], axis=1)

In [75]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8603 entries, 324857 to 56443
Data columns (total 20 columns):
title                    8603 non-null object
belongs_to_collection    1516 non-null object
budget                   2529 non-null float64
genres                   8138 non-null object
imdb_id                  8603 non-null object
overview                 8529 non-null object
revenue                  2020 non-null float64
tagline                  4544 non-null object
vote_average             7944 non-null float64
vote_count               7956 non-null float64
runtime                  8118 non-null float64
popularity               8603 non-null float64
production_companies     6793 non-null object
director_name            4691 non-null object
director_gender          4691 non-null float64
leading_actor_name       4314 non-null object
supporting_actor_name    4607 non-null object
year                     8557 non-null float64
month_of_year            8557 non-null object
day_o

In [61]:
# Function to create a new SQLite3 Database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [62]:
# Use create_connection function to create Database
create_connection("/Users/florencialeoni/Documents/flatironDS/Mod_1_FP/mod1_movies/movies.db")

In [63]:
# Create a connection object using the connect function 
# To establish a connection with "movies.db"
connection = sqlite3.connect("movies.db")
# Create a cursor object by calling the cursor method
cursor = connection.cursor()

In [65]:
# Write records stored in the imdb_df DataFrame to a SQL database
imdb_df.to_sql("imdb_movies", con = connection)

In [64]:
# Write records stored in the tmdb_df DataFrame to a SQL database
tmdb_df.to_sql("tmdb_movies", con = connection)